In [92]:
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WikipediaLoader,TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage,SystemMessage,AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableMap,RunnableLambda,RunnableParallel,RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from pydantic import BaseModel,Field
from dotenv import load_dotenv
load_dotenv()

llm_model = ChatGoogleGenerativeAI(model = 'gemini-2.0-flash',max_retries=2)
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2') ##embedding model

In [3]:
documents = WikipediaLoader(
    query='Machine Learning',
    lang='en',
    load_max_docs=15
).load()

print(f"Number of docs loaded:\n{len(documents)}")
print(f"Document metadata:\n{documents[0].metadata}")
print(f"Document Page content:\n{documents[0].page_content}")

Number of docs loaded:
15
Document metadata:
{'title': 'Machine learning', 'summary': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.\nStatistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via uns

In [5]:
chunker = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
    separators=["\n\n","\n"," "]
)
doc_chunks = chunker.split_documents(documents=documents)

print(f"Number of chunks loaded:\n{len(doc_chunks)}")
print(f"Chunk metadata:\n{doc_chunks[0].metadata}")
print(f"Chunk Page content:\n{doc_chunks[0].page_content}")

Number of chunks loaded:
197
Chunk metadata:
{'title': 'Machine learning', 'summary': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.\nStatistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via uns

In [8]:
persist_dir = './chromadb'
vector_store = Chroma.from_documents(
    documents=doc_chunks,
    embedding=embedding_model,
    collection_name='query_decomp1',
    persist_directory=persist_dir
)
print(f"Number of vectors created in the store: {vector_store._collection.count()}")

retriever = vector_store.as_retriever(search_type='mmr',
                                      search_kwargs={'k':3})

Number of vectors created in the store: 197


In [33]:
### query decomposition chain ##

def format_questions_list(questions):

    question_dict = dict(questions)
    questions_list = list(question_dict.values())
    return questions_list
    

class decomposition_resp(BaseModel):
    subques1:str = Field(...,description='SubQuestion number 1')
    subques2:str = Field(...,description='SubQuestion number 2')
    subques3:str = Field(...,description='SubQuestion number 3')

def decompositionChain(query):
    system_prompt = SystemMessagePromptTemplate.from_template(
        '''You are an intelligent assistant, which understands the human query nuances.
    Decompose the question into 3 sub-questions for better retrieval and better coverage of the human query'''
    )

    human_prompt = HumanMessagePromptTemplate.from_template(
        '''Human Query : {query}'''
    )

    chat_prompt = ChatPromptTemplate.from_messages([system_prompt,human_prompt])
    decomposition_chain = chat_prompt | llm_model.with_structured_output(decomposition_resp) | format_questions_list

    return decomposition_chain.invoke({'query':query})


In [ ]:
## question retriever chain ###
from typing import List

def synthesizer(query,subans1,subans2,subans3):

    rag_instruction_prompt = ChatPromptTemplate.from_template(
    '''
    You are an intelligent synthesizer, based on following query & retrieved answers,
    create a combined answer, which  represents all the three answers, but keep the combined output compact.

    query : {query}
    answer1 : {subans1},
    answer2 : {subans2},
    answer3 : {subans3}
    '''
    )

    syn_chain = rag_instruction_prompt | llm_model | StrOutputParser()
    return syn_chain.invoke({'query':query,'subans1':subans1,'subans2':subans2,'subans3':subans3})



def format_docs(docs)->str:
    '''Creating a context from chunks'''
    return "\n\n".join(doc.page_content for doc in docs)



def rag_chain(query):
 
    subquestions = decompositionChain(query)
    #print(subquestions[0])

    rag_instruction_prompt = ChatPromptTemplate.from_template(
    '''
    You are an intelligent assistant, based on following  retrieved context documents,
    answer the question in a concise and streamlined manner in about 4-5 sentences.

    Context : {context},
    question : {query}
    '''
    )

    rag_chain = (
    RunnableParallel(branches={
            "sub_question1_chain": {'context': lambda _:format_docs(retriever.invoke(subquestions[0])) , "query": lambda _: subquestions[0]} | rag_instruction_prompt | llm_model | StrOutputParser(),
            "sub_question2_chain": {'context': lambda _:format_docs(retriever.invoke(subquestions[1])) , "query": lambda _: subquestions[1]} | rag_instruction_prompt | llm_model | StrOutputParser(),
            "sub_question3_chain": {'context': lambda _:format_docs(retriever.invoke(subquestions[2])) , "query": lambda _: subquestions[2]} | rag_instruction_prompt | llm_model | StrOutputParser(),
        }) | RunnableLambda(lambda x : synthesizer(query, x['branches']['sub_question1_chain'],
                                                   x['branches']['sub_question2_chain'],
                                                   x['branches']['sub_question3_chain']))
                                                   )
    

    return rag_chain.invoke(None)
    



In [108]:
rag_chain("Explain fundamentals of Machine Learning")

'Machine learning (ML), a subfield of AI, empowers computers to learn from data without explicit programming by building models from training data to make predictions. This involves selecting a learning algorithm (e.g., support-vector machines, decision trees) and determining the structure of the learned function. Deep learning, a type of ML, uses multi-layered neural networks. Model building includes steps like tuning control parameters using validation sets or cross-validation. Key considerations include model robustness (addressed by adversarial ML) and the need for transparency and interpretability in decision-making (addressed by explainable AI or XAI).'